# COMS 4995_002 Deep Learning Assignment 1
Due on Thursday, Feb 8, 11:59pm

This assignment can be done in groups of at most 2 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Apoorv Purwar, ap3644

Member 2: Shreyas Mundhra, ssm2211

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [39]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.num_layers = len(layer_dimensions) #Length of tuple is the number of layers
        self.parameters = {} 
        for i in range(1, self.num_layers):
            curr_layer_dim = layer_dimensions[i]
            prev_layer_dim = layer_dimensions[i-1]
#             # Random Initialisation of bias
#             bias = (np.random.uniform()/10)*np.ones((curr_layer_dim,1)) + 0.01
#             # Random Initialisation of weights
#             weight = np.random.normal(0,10*np.random.uniform(),(curr_layer_dim, prev_layer_dim)) 


            epsilon = np.sqrt(2.0 / (curr_layer_dim + prev_layer_dim))
            weight = np.random.randn(curr_layer_dim, prev_layer_dim) * epsilon
            bias = np.zeros((curr_layer_dim, 1)) + 0.01 # added to make sure relus fire in the start
            self.parameters[i] = [weight, bias]
        self.drop_prob = 1 - drop_prob
        self.reg_lambda = reg_lambda  
        
    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
#         print("affine W shape: ",W.shape)
#         print("affine forward A shape: ",A.shape)
#         y = np.matmul(W,A) + b
        y = W.dot(A) + b
#         print("unique y: ",np.unique(y))
        cache = [A,y,W]
        return y, cache
        

    def activationForward(self, A, activation="relu"):
        """¬
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        activation = self.relu(A)  # since no other activation fn exists
        return activation


    def relu(self, X):
        A = np.maximum(0, X)
        return A
            
    def dropout(self, A, prob):
        M = (np.random.rand(*A.shape)<prob)/prob
        A = A*M
        return A, M
           
       # param A: 
       # :param prob: drop prob
       # :returns: tuple (A, M) 
       #     WHERE
       #     A is matrix after applying dropout
       #     M is dropout mask, used in the backward pass
      

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        cache = {}
        prev_activation = X 
        for i in range(1, self.num_layers-1):
            nodes = layer_dimensions[i]
            curr_activations = np.zeros((nodes,X.shape[1])) # Get the column count of X - Can we comment this?
            curr_activations, cache[i] = \
            self.affineForward(prev_activation, self.parameters[i][0], self.parameters[i][1])
            curr_activations = self.activationForward(curr_activations)
            prev_activation = curr_activations
            
        
        nodes = layer_dimensions[self.num_layers-1]
        curr_activations = np.zeros((nodes,X.shape[1])) # Get the column count of X - Can we comment this?
        curr_activations, cache[self.num_layers-1] = \
        self.affineForward(prev_activation, self.parameters[self.num_layers-1][0], self.parameters[self.num_layers-1][1])
        
        AL = curr_activations

        return AL, cache


    
    
    
#     def forwardPropagation(self, X):
#         """
#         Runs an input X through the neural network to compute activations
#         for all layers. Returns the output computed at the last layer along
#         with the cache required for backpropagation.
#         :returns: (tuple) AL, cache
#             WHERE 
#             AL is activation of last layer
#             cache is cached values for each layer that
#                      are needed in further steps
#         """
#         cache = {} #cache of caches from each layer
# #         drop_prob = self.drop_prob
        
#         for i in range(1, self.num_layers - 1):
# #             W = "W" + str(i)
# #             b = "b" + str(i)
# #             linearCache = 'linear' + str(i)
# #             activationCache = 'activation' + str(i)
# #             dropoutCache = 'dropout' + str(i)
#             curr_activations, cache[i] = self.affineForward(X, self.parameters[i][0], self.parameters[i][1])
# #             cache[linearCache] = cacheLinearValue
#             curr_activations = self.activationForward(X, "relu")
# #             cache[activationCache] = cacheActivationValue
# #             if (drop_prob > 0):
# #                 X, cacheDropoutValue = self.dropout(X, drop_prob)
# #                 cache[dropoutCache] = cacheDropoutValue

# #         W = "W" + str(self.num_layers - 1)
# #         b = "b" + str(self.num_layers - 1)
#         AL, cache[self.num_layers-1] = self.affineForward(X, self.parameters[self.num_layers-1][0], self.parameters[self.num_layers-1][1])
# #         cache['linear' + str(self.num_layers - 1)] = cacheLinearValue
        
#         return AL, cache
    
    def softmax(self, X):
        return np.exp(X) / np.sum(np.exp(X),axis=0) #changes input into probabilities

    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        
#         print("Unique AL: ",np.unique(AL))
        numberSamples = AL.shape[1]
        AL = AL - np.max(AL, axis=0, keepdims=True) #shifted values to make max value 0 so that exp values don't explode
        AL_prob = self.softmax(AL)
        correct_label_prob = AL_prob[y, np.arange(numberSamples)]
        logProb = np.log(correct_label_prob)
        cost = -np.sum(logProb) / numberSamples
        
        #add regularization
        if self.reg_lambda > 0:
            for i in range(1, self.num_layers):
                W = "W" + str(i)
                cost += 0.5*self.reg_lambda*np.sum(np.square(self.parameters[W])) #multiplied by half to help with gradient
                #gradient from regularization added at the end of backprop
        
        # gradient of cost
        AL_prob[y, np.arange(numberSamples)] -= 1
        dAL = AL_prob/float(numberSamples)
        return cost, dAL
    
    
    
            
#     def costFunction(self, AL, y):
#         """
#         :param AL: Activation of last layer, shape (num_classes, S)
#         :param y: labels, shape (S)
#         :param alpha: regularization parameter
#         :returns cost, dAL: A scalar denoting cost and the gradient of cost
#         """
#         S = len(y)
#         num_classes = AL.shape[0]
# #         AL[AL == 0] = 0.00001
        
#         softmax_out = np.exp(AL, dtype=float)
#         softmax_sums = np.sum(softmax_out,axis=0)
#         softmax_out = softmax_out/softmax_sums
#         softmax_out_T = softmax_out.T
        
#         cost_array = np.zeros(S)
#         for i in range(0,len(y)):
#             cost_array[i] = -np.log(softmax_out_T[i,y[i]])
# #         print(cost_array.shape)
#         cost = np.sum(cost_array)
#         cost = cost/float(S)
       
#         dAL = np.zeros(softmax_out_T.shape)
# #         print("Unique dAL 1: " + str(np.unique(dAL)))
# #         print("softmax_out_T shape: ", softmax_out_T[1], softmax_out_T[y].shape) #100*10
#         for i in range(0,len(y)):
#             dAL[i,y[i]]  = -(1/(softmax_out_T[i,y[i]]))
# #         print("Unique dAL 2: " + str(np.unique(dAL)))
#         softmax_out_T = np.multiply(softmax_out_T, np.subtract(np.ones(softmax_out_T.shape),softmax_out_T))
#         dAL = dAL*softmax_out_T
#         dAL = dAL.T
        
        
        
# #         AL_prob = self.softmax(AL)
# #         # gradient of cost
# #         AL_prob[y, np.arange(S)] -= 1
# #         dAL = AL_prob/float(S)
        
# #         dAL = np.sum(dAL_matrix,axis=1)
        
        
        
# #         one_hot_y = one_hot(y)
        
# #         num_classes = one_hot_y.shape[0]
# #         S = one_hot_y.shape[1]
                
# #         # compute loss
# #         cost_matrix = one_hot_y - AL
# #         cost = np.linalg.norm(cost_matrix,ord=2)
        
#         if self.reg_lambda > 0:
#             pass
#             # add regularization
       
        
#         # gradient of cost
# #         dAL_matrix = -cost_matrix
# #         dAL = dAL_matrix[:,0]
# #         for j in range(1,S):
# #             dAL = dAL + dAL_matrix[:,j]
            
# #         dAL = dAL/float(S)
        
    
#         print("Unique dAL: ",np.unique(dAL))
        
#         return cost, dAL

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        A = cache[0]
        y = cache[1]
        W = cache[2]
        
#         print("affine backward A shape: ",A.shape)
        
        S = y.shape[1]
        
#         A = A_matrix[:,0]
#         for j in range(1,S):
#             A = A + A_matrix[:,j]
#         A = A/float(S)
#         A = A.reshape((len(A),1))
        
        dy = self.activationBackward(dA_prev, cache)
#         print("Unique dy: " + str(np.unique(dy)))
        
#         dA = np.matmul(W.transpose(),dy)
        dA = W.T.dot(dy)
        
#         dW = np.matmul(dy, A.transpose())
        dW = dy.dot(A.T)
#         dW = dW/float(S)
        db = np.sum(dy, axis=1)
#         db = db/float(S)
        db = db.reshape((len(db), 1))
#         print("affine back dy shape: ",dy.shape)
#         print("affine back db shape: ",db.shape)
        return dA, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        dx = self.relu_derivative(dA,cache[1])
        return dx

        
    def relu_derivative(self, dx, cached_x):
        nodes, S = cached_x.shape
        cached_x_derivative = np.zeros((nodes,S))
        
        cached_x_derivative[cached_x > 0] = 1
        
#         for i in range(0, nodes):
#             for j in range(0, S):
#                 if cached_x[i,j] > 0:
#                     cached_x_derivative[i,0] = cached_x_derivative[i,0] + 1
                    
#         cached_x_derivative = cached_x_derivative/float(S)
#         dx = np.multiply(dx.T,cached_x_derivative.T).T
        dx = np.multiply(dx,cached_x_derivative)
#         print("relu derivative dx shape: ",dx.shape)
        return dx

    def dropout_backward(self, dA, cache):

        return dA

    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {}
        
        S = len(Y)
        
        
        
        A = cache[self.num_layers-1][0]
#         print("backprop A shape: ",A.shape)
        W = cache[self.num_layers-1][2]
#         dA = np.matmul(W.transpose(),dAL)
        dA = W.T.dot(dAL)
        
#         dW = np.matmul(dAL, A.transpose())
        dW = dAL.dot(A.T)
#         dW = dW/float(S)
        db = np.sum(dAL, axis=1)
#         db = db/float(S)
        db = db.reshape((len(db), 1))
        gradients[self.num_layers-1] = [dW,db]
        dA_prev = dA
        
        
        
        
        
#         dA_prev = dAL
#         print("Unique dAL: " + str(np.unique(dAL)))
        
        for l in range(self.num_layers-2,0,-1):
            dA, dW, db = self.affineBackward(dA_prev, cache[l])
#             print("Unique db: " + str(np.unique(db)))
            
            gradients[l] = [dW,db]
#             print("backprop dA shape: ",dA.shape)
#             print("backprop dW shape: ",dW.shape)
#             print("backprop db shape: ",db.shape)
            dA_prev = dA
            
        if self.drop_prob > 0:
                #call dropout_backward
                pass
           
            
        if self.reg_lambda > 0:
            # add gradients from L2 regularization to each dW
            pass
        
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
#         print("Unique gradients: " + str(np.unique(np.asarray(gradients))))
        for i in range(1, self.num_layers):
            self.parameters[i][0] = self.parameters[i][0] - alpha*gradients[i][0]
            self.parameters[i][1] = self.parameters[i][1] - alpha*gradients[i][1]

    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        # TODO: divide into train and validation set
        # TODO:  print train and val set accuracies
        
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        
        S = len(y)
        train_split_percent = 90
        
        num_train = train_split_percent*S/100
        num_val = S - num_train
        
        X_train = X[:, :num_train]
        y_train = y[:num_train]
        X_val = X[:, num_train:]
        y_val = y[num_train:]
        
        X_batch, y_batch = self.get_batch(X_train,y_train,batch_size)
        numBatches = len(y_batch)
        
        for i in range(0, iters):
#             for j in range(0, numBatches):
            # get minibatch
            X_mini = X_batch[i % numBatches]
            y_mini = y_batch[i % numBatches]

            # forward prop
            AL, cache = self.forwardPropagation(X_mini)

            # compute loss
            cost, dAL = self.costFunction(AL, y_mini)

            # compute gradients
            gradients = self.backPropagation(dAL, y_mini, cache)

            # update weights and biases based on gradient
            self.updateParameters(gradients, alpha)

            if i % print_every == 0:
                y_pred_train = self.predict(X_train)
#                 cost_train, dAL = self.costFunction(y_pred_train, y_train)
                
                y_pred_val = self.predict(X_val)
#                 cost_val, dAL = self.costFunction(y_pred_val, y_val)
                
                train_preds = y_pred_train
                train_accuracy = 0
                for i in range(0, len(y_train)):
                    if y_pred_train[y_train[i],i] == 1:
                        train_accuracy = train_accuracy + 1
                train_accuracy = train_accuracy*100.0/num_train
                
                val_accuracy = 0
                for i in range(0, len(y_val)):
                    if y_pred_val[y_val[i],i] == 1:
                        val_accuracy = val_accuracy + 1
                val_accuracy = val_accuracy*100.0/num_val
                
                # print cost, train and validation set accuracies
                print("(Cost, training accuracy, val accuracy: )", cost, train_accuracy, val_accuracy)
                
    def predict(self, X):
        """
        Make predictions for each sample
        """         
        AL, cache = self.forwardPropagation(X)
        
        S = AL.shape[1]
        y_pred = np.zeros(S, dtype=int)
        
        for j in range(0, S):
            y_pred[j] = np.argmax(AL[:,j])
        
        return one_hot(y_pred)

    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        n = len(y)
        
        X_batch = []
        y_batch = []
        for start in range(0,n-batch_size,batch_size):
            X_batch.append(X[:,start:start+batch_size])
            y_batch.append(y[start:start+batch_size])
            
        return X_batch, y_batch

In [13]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [14]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [5]:
# Load the data
data_root_path = './cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'horse': 7, 'automobile': 1, 'deer': 4, 'dog': 5, 'frog': 6, 'cat': 3, 'truck': 9, 'ship': 8, 'airplane': 0, 'bird': 2}
Loaded 10000/17232
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [40]:
# layer_dimensions = [X_train.shape[0], 3, 10]  # including the input and output layers
# NN = NeuralNetwork(layer_dimensions)
# NN.train(X_train, y_train, alpha=0.0001,print_every=100)
layer_dimensions = [X_train.shape[0], 512, 256, 128, 10]  # including the input and output layers
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train, iters=1000, alpha=0.0001, batch_size=200, print_every=200)

('(Cost, training accuracy, val accuracy: )', 2.3602306851803814, 0.05803456280629352, 0.2900232018561485)
('(Cost, training accuracy, val accuracy: )', 2.3245787463622518, 0.06448284756254836, 0.580046403712297)


KeyboardInterrupt: 

In [ ]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Improving the performance

In [27]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0, reg_lambda=0)
NN2.train(X_train, y_train, iters=1000, alpha=0.00001, batch_size=1000, print_every=10)

('(Cost, training accuracy, val accuracy: )', 2.3025850929940459, 0.03868970853752902, 0.23201856148491878)
('(Cost, training accuracy, val accuracy: )', 2.3025850929940459, 0.03224142378127418, 0.23201856148491878)
('(Cost, training accuracy, val accuracy: )', 2.3025850929940459, 0.03224142378127418, 0.1740139211136891)
('(Cost, training accuracy, val accuracy: )', 2.3025850929940459, 0.025793139025019344, 0.1740139211136891)


KeyboardInterrupt: 

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')

Write down results for Part 2 here:
...

In [126]:
import numpy as np
a = np.array([0.7,0.8,0.6,0.3]).reshape(2,2)
# = np.arange(2*4*2).reshape((2,4,2))
# = np.arange(2*2*4).reshape((2,2,2))
#print("A = ", (np.random.rand(*a.shape)<0.3)/0.3)
#print("B = ", b)

#print("Dot = ", np.dot(a,b))

#print("Matmul = ", np.matmul(a,b))

#np.random.seed(0);
#np.random.rand(4)
#10*np.random.uniform()
#np.random.normal(0,2,(10,10))


0.15606064446828216

In [152]:
import numpy as np
bias = (np.random.uniform()/10)*np.ones((3,1)) 
# Random Initialisation of weights
weight = np.random.normal(0,10*np.random.uniform(),(3, 5)) 
param = [weight, bias]
print("weight", weight)
print("bias", bias)
print("param",param)

weight [[  7.82934072 -10.86774528   5.20450091  -5.36508355  -9.3388032 ]
 [ -5.70175423   5.57958156  -2.6834555   -1.45635979   2.89883828]
 [  3.90676038  -5.48807876   0.28179786  -0.53944225  -1.81890684]]
bias [[ 0.06658591]
 [ 0.06658591]
 [ 0.06658591]]
param [array([[  7.82934072, -10.86774528,   5.20450091,  -5.36508355,  -9.3388032 ],
       [ -5.70175423,   5.57958156,  -2.6834555 ,  -1.45635979,
          2.89883828],
       [  3.90676038,  -5.48807876,   0.28179786,  -0.53944225,
         -1.81890684]]), array([[ 0.06658591],
       [ 0.06658591],
       [ 0.06658591]])]
